In [ ]:
## Notebook to evaluate and store the results from ggcnn for a sample imageset
## Last Trial Run: 7th April 2021

In [2]:
import os 
import glob
import sys 
import torch
import numpy as np
import torch.utils.data 
import imageio
import pandas as pd
import time

In [3]:
ggcnn_dir = "/media/gvk/GVK_Elements/github_repo/ggcnn_stacking"
fadi_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Experts/Generative/pretrained_weights"
sys.path.append(ggcnn_dir)
sys.path.append(fadi_dir)
from models.common import post_process_output
from utils.dataset_processing import evaluation, grasp 
from utils.data import get_dataset, cornell_data, cornell_data_neg
from utils.data.cornell_data import CornellDataset
from utils.data.cornell_data_neg import CornellDataset_Neg
from utils.dataset_processing.image import DepthImage

In [4]:
## HYPERPARAMETERS:
NO_GRASPS = 1

In [122]:
## Model Loading:
MODEL_NAME = 'FadiColorNet'
GRASP_TYPE = "Positive"
MODEL_DIR = os.path.join(fadi_dir, "color", "epoch_37_iou_0.80")
IOU_NAME = MODEL_NAME + "_IoU_AllGrasps_FullCornell.csv"
depth_net = torch.load(MODEL_DIR)
net = depth_net
include_depth = False
include_rgb = True
print(IOU_NAME)

FadiColorNet_IoU_AllGrasps_FullCornell.csv


In [123]:
## Considering one folder!
dataset_path = "/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/*/"
positive_data = CornellDataset(dataset_path, include_depth = False, include_rgb = True)


In [124]:
positive_data.grasp_files[0]

'/media/gvk/GVK_Elements/WPI/Spring2021/DR_Grasping/CurrentFocus/Cornell_Dataset/Full_Cornell/data01/01/pcd0100cpos.txt'

In [125]:
len(positive_data.depth_files)

885

In [126]:
## For the purpose of comparing images at a later stage.
positive_comp = positive_data
positive_data = torch.utils.data.DataLoader(positive_data, batch_size = 1, shuffle = False)

In [127]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [128]:
# x, y, didx, rot, zoom = next(iter(positive_data))

In [129]:
## For all the images within the Cornell Dataset!

results = {'correct': 0, 'failed': 0}
start_cell = time.time()
iou_list = []
with torch.no_grad():

    for img_idx, (x, y, didx, rot, zoom) in enumerate(positive_data):
        xc = x.to(device)
        yc = [yi.to(device) for yi in y]
        lossd = net.compute_loss(xc, yc)    ## Ensure that your outputs are from a dataloader!

        ## Getting the Quality, Angle and Width Images!
        q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])

        ## Getting the ground truth Bounding Boxes for the particular image within Cornell Dataset - Refer dataset_preprocessing.evaluation.py for more details.
        groundTruth_bbs = positive_data.dataset.get_gtbb(didx, rot, zoom)
        if not isinstance(groundTruth_bbs, grasp.GraspRectangles):
            groundTruth_bbs = grasp.GraspRectangles.load_from_array(groundTruth_bbs)
        
        grasp_predict = grasp.detect_grasps(q_img, ang_img, width_img, no_grasps = NO_GRASPS)
        
        iou_dict = {}
        ## For each grasp available:
        for g_idx, model_grasp in enumerate(grasp_predict):
            for gt_idx, gt_bb in enumerate(groundTruth_bbs):
                ## Calculate IoU:
                grasp_predict = model_grasp.as_gr

                key_name = "_".join(("ModelGrasp", str(g_idx), "GT_Grasp", str(gt_idx)))
                ## Detecting the IoU between the Grasp and Ground Truth Bounding Box
                iou_dict[key_name] = grasp_predict.iou(gt_bb)
        
        ## Saving the IoU generated!
        ## Just getting the pcd name:
        
        img_id = positive_comp.depth_files[img_idx].split("/")[-1]  ## Now its pcdxxxx.tiff
        img_id = str(img_id).split(".")[0][:-1]                 ## Now its pcdxxxx
        for key_grasp, value_grasp in iou_dict.items():
            grasp_result = value_grasp > 0.25
            iou_list.append([img_id, key_grasp, value_grasp, grasp_result, didx.item(), zoom.item(), rot.item()])
print("Cell Time: ", time.time() - start_cell) ## ~52s

Cell Time:  108.09414744377136


In [130]:
iou_list[0]

['pcd0100', 'ModelGrasp_0_GT_Grasp_0', 0.2835249042145594, True, 0, 1.0, 0.0]

In [131]:
iou_pd = pd.DataFrame(iou_list)
iou_pd.columns = ['Image_ID', 'Grasp_ID', 'IoU', 'Grasp_Result', 'Didx_Image', 'Image_Zoom', 'Image_Rot']
iou_pd['Grasp_Type'] = +1
iou_pd

,Image_ID,Grasp_ID,IoU,Grasp_Result,Didx_Image,Image_Zoom,Image_Rot,Grasp_Type
0,pcd0100,ModelGrasp_0_GT_Grasp_0,0.283525,True,0,1.0,0.0,1
1,pcd0100,ModelGrasp_0_GT_Grasp_1,0.448869,True,0,1.0,0.0,1
2,pcd0100,ModelGrasp_0_GT_Grasp_2,0.048695,False,0,1.0,0.0,1
3,pcd0100,ModelGrasp_0_GT_Grasp_3,0.000000,False,0,1.0,0.0,1
4,pcd0101,ModelGrasp_0_GT_Grasp_0,0.000000,False,1,1.0,0.0,1
...,...,...,...,...,...,...,...,...
5105,pcd1034,ModelGrasp_0_GT_Grasp_0,0.000000,False,884,1.0,0.0,1
5106,pcd1034,ModelGrasp_0_GT_Grasp_1,0.052144,False,884,1.0,0.0,1
5107,pcd1034,ModelGrasp_0_GT_Grasp_2,0.000000,False,884,1.0,0.0,1
5108,pcd1034,ModelGrasp_0_GT_Grasp_3,0.336987,True,884,1.0,0.0,1


In [132]:
# iou_pd = pd.DataFrame(iou_list)
# iou_pd.columns = ['Image_ID', 'Grasp_ID', 'IoU', 'Grasp_Result', 'Didx_Image', 'Image_Zoom', 'Image_Rot']
# iou_pd['Grasp_Type'] = +1
# iou_pd

In [133]:
iou_pd.to_csv(IOU_NAME, sep = ',', index = False)
IOU_NAME

'FadiColorNet_IoU_AllGrasps_FullCornell.csv'

In [134]:
STOP

NameError: name 'STOP' is not defined

In [36]:
# print(len(iou_quality.keys()))

In [37]:
# ## To get the image id
# img_id = test_comp.rgb_files[1].split("/")[-1]
# depth_id = str(img_id).split('.')[0][:-1]
# print(depth_id)

# rgb_id = str(img_id).split('.')[0][:-1]
# print(rgb_id)

In [38]:
# ## Saving the Collected Results!
# import pickle
# model_name = "fadi_depth"
# save_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/"
# first_entry, last_entry = list(iou_quality.keys())[0], list(iou_quality.keys())[-1]
# fname = "_".join((model_name, first_entry, last_entry+".txt"))
# fdir = os.path.join(save_dir, fname)
# with open(fdir, 'wb') as file_pickle:
#     pickle.dump(iou_quality, file_pickle)

In [39]:
#  Trial for each test image
## Test Worked: 11th May 2021
## To detect whether the grasp worked for all possible grasps!
start_cell = time.time()
results = {'correct': 0, 'failed': 0}
iou_quality = {}
with torch.no_grad():

    ## For each test image
    ## Just testing for one single image
    x, y, didx, rot, zoom =  next(iter(positive_data))
    img_idx = 0
    xc = x.to(device)
    yc = [yi.to(device) for yi in y]
    lossd = net.compute_loss(xc, yc)    ## Ensure that your outputs are from a dataloader!

    ## Getting the Quality, Angle and Width Images!
    q_img, ang_img, width_img = post_process_output( lossd['pred']['pos'], lossd['pred']['cos'], lossd['pred']['sin'], lossd['pred']['width'])

    ## Getting the ground truth Bounding Boxes for the particular image within Cornell Dataset - Refer dataset_preprocessing.evaluation.py for more details.
    groundTruth_bbs = positive_data.dataset.get_gtbb(didx, rot, zoom)
    if not isinstance(groundTruth_bbs, grasp.GraspRectangles):
        groundTruth_bbs = grasp.GraspRectangles.load_from_array(groundTruth_bbs)
    
    grasp_predict = grasp.detect_grasps(q_img, ang_img, width_img, no_grasps = NO_GRASPS)
    
    iou_dict = {}
    ## For each grasp available:
    for g_idx, model_grasp in enumerate(grasp_predict):
        for gt_idx, gt_bb in enumerate(groundTruth_bbs):
            ## Calculate IoU:
            grasp_predict = model_grasp.as_gr

            key_name = "_".join(("ModelGrasp", str(g_idx), "GT_Grasp", str(gt_idx)))
            ## Detecting the IoU between the Grasp and Ground Truth Bounding Box
            iou_dict[key_name] = grasp_predict.iou(gt_bb)
    
    ## Saving the IoU generated!
    ## Just getting the pcd name:
    iou_list = []
    img_id = positive_comp.depth_files[img_idx].split("/")[-1]  ## Now its pcdxxxx.tiff
    img_id = str(img_id).split(".")[0][:-1]                 ## Now its pcdxxxx
    for key_grasp, value_grasp in iou_dict.items():
        grasp_result = value_grasp > 0.25
        iou_list.append([img_id, key_grasp, value_grasp, grasp_result, didx.item(), zoom.item(), rot.item()])


    # ## Now to compare and create a dictionary that stores the IoU given image name
    # x_comp, _, _, _, _ = test_comp[idx]
    # #print(torch.allclose(torch.tensor(x_comp), x))
    # if torch.allclose(torch.tensor(x_comp), x):
    #     img_id = test_comp.depth_files[0].split("/")[-1]

    #     ## We cannot use an image!
    #     #quality_dict[str(img_id)] = q_img
    #     image_id = str(img_id).split('.')[0][:-1]

    #     # iou_quality[image_id] = [int(s), np.max(q_img).astype('float32')]
    # else:
    #     print("Dataloader and Comparison X didn't match")

print("Cell Time: ", time.time() - start_cell)

Cell Time:  0.06836748123168945
